# 检查transport reaction 

In [12]:
import pandas as pd

# 读取xlsx文件和tsv文件
xlsx_file = '3.Add exchange reactions and biomass reaction/14067_rxn_3.xlsx'  # 替换为你的xlsx文件路径
tsv_file = '4. add linker/modelseed_reactions.tsv'  # 替换为你的tsv文件路径
df_xlsx = pd.read_excel(xlsx_file)

# 筛选出source列为"add by Manual"的行
df_filtered = df_xlsx[df_xlsx['source'] == 'add by Manual']

# 使用.loc来明确修改操作
df_filtered.loc[:, 'ID'] = df_filtered['ID'].str.replace('_c0$', '', regex=True)

# 读取tsv文件
df_tsv = pd.read_csv(tsv_file, sep='\t')

# 合并两个数据集，匹配ID列
df_merged = pd.merge(df_filtered, df_tsv, left_on='ID', right_on='id', how='inner')

# 筛选出is_transport不为0的行
df_transport = df_merged[df_merged['is_transport'] != 0]

# 输出匹配的ID格式为rxn{ID}
for _, row in df_transport.iterrows():
    print(f"rxn{{{row['ID']}}}")

# 提取 biomass中的代谢物，将其转换为modelseedid

In [18]:
import pandas as pd
import re

# 文件路径
file_path = '3.Add exchange reactions and biomass reaction/biomass equation.xlsx'  # 替换为你的实际文件路径
output_path = 'biomass_metabolites.xlsx'  # 输出文件路径

# 读取 Excel 文件的第二个表格
sheet_name = 1  # 第二个工作表（索引从 0 开始）
data = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

# 正则表达式匹配代谢物名称（忽略数字系数）
reaction_pattern = re.compile(r"\b[a-zA-Z_]+[a-zA-Z0-9_-]*\b")

# 提取代谢物
all_metabolites = set()

for row in data.itertuples(index=False):
    if len(row) > 1:
        reaction = row[1]  # 假设反应公式在第二列
        if isinstance(reaction, str):
            # 分割反应物和产物
            if "-->" in reaction:
                reactants, products = reaction.split("-->", 1)
                reactants_metabolites = reaction_pattern.findall(reactants)
                products_metabolites = reaction_pattern.findall(products)
                all_metabolites.update(reactants_metabolites)
                all_metabolites.update(products_metabolites)

# 转换为 DataFrame 并去重
metabolites_df = pd.DataFrame(sorted(all_metabolites), columns=["Metabolite"])

# 输出到新的 Excel 文件
metabolites_df.to_excel(output_path, index=False)

print(f"提取完成，唯一代谢物列表（无数字）已保存到 {output_path}")


TypeError: expected string or bytes-like object, got 'float'

# 手动替换modelseedid，不建议使用下面的代码

In [25]:
import pandas as pd
import re

# 读取代谢物和rxn id的对应关系
metabolites_df = pd.read_excel('[完整版已整合进bioeq file]biomass_metabolites.xlsx')
# 读取反应式
reactions_df = pd.read_excel('biomass equation.xlsx', sheet_name='Sheet2')  # 假设反应式在第一个工作表

# 创建一个字典，用于存储代谢物和rxn id的对应关系
metabolite_to_rxn = metabolites_df.set_index('Metabolite')['rxn id'].to_dict()

# 编译正则表达式
reaction_pattern = re.compile(r"\b[a-zA-Z_]+[a-zA-Z0-9_-]*\b")

# 清理和替换反应物
def replace_metabolite(equation, metabolite_to_rxn):
    # 确保equation是字符串类型
    equation = str(equation)
    
    # 使用正则表达式提取反应物
    matches = reaction_pattern.findall(equation)
    
    # 替换反应物
    for match in matches:
        if match in metabolite_to_rxn and pd.notnull(metabolite_to_rxn[match]):
            # 确保匹配项前后没有其他字母或数字
            start_idx = equation.find(match)
            end_idx = start_idx + len(match)
            if (start_idx == 0 or not equation[start_idx-1].isalnum()) and \
               (end_idx == len(equation) or not equation[end_idx].isalnum()):
                equation = equation.replace(match, metabolite_to_rxn[match])
    
    # 将替换后的反应物放回反应式
    return equation

# 应用替换函数到每个反应式
reactions_df['Replaced Equation'] = reactions_df['Equation'].apply(lambda x: replace_metabolite(x, metabolite_to_rxn))

# 保存替换后的结果到新的Excel文件
reactions_df.to_excel('[完整版已整合进bioeq file]replaced_biomass_equations.xlsx', index=False)

print("替换完成，结果已保存到 [完整版已整合进bioeq file]replaced_biomass_equations.xlsx")

替换完成，结果已保存到 replaced_biomass_equations.xlsx


# 调转反应方向

In [13]:
import pandas as pd

# 读取你的表格文件
df = pd.read_excel("14067_rxn_3.xlsx")  # 替换为你的文件名
reaction_column = "Definition"  # 替换为反应所在列的名称

# 定义一个函数，仅处理 '<=' 符号，将其转换为 '=>'
def reverse_reaction(reaction):
    if not isinstance(reaction, str):  # 检查是否为字符串，非字符串直接返回原值
        return reaction
    # 确保只处理包含 '<=' 的情况
    if '<=' in reaction and '<=>' not in reaction:
        # 分割 '<=' 并反转方向
        products, reactants = reaction.split('<=')
        # 去除多余空格并拼接成标准化形式
        return f"{reactants.strip()} => {products.strip()}"
    return reaction  # 其他情况保持不变

# 应用反应方向调整函数，仅对 '<=' 的反应式进行转换
df[reaction_column] = df[reaction_column].apply(reverse_reaction)

# 打印调整后的反应式
print("调整后的反应式：")
print(df[reaction_column])

# 保存到新的文件
output_file = "3.Add exchange reactions and biomass reaction/14067_rxn_3.xlsx"
df.to_excel(output_file, index=False)

print(f"处理完成，结果已保存到 {output_file}")

调整后的反应式：
0       (1) H2O_c0[c0] + (1) PPi_c0[c0] => (2) Phospha...
1       (1) H2O_c0[c0] + (3) H+_c0[c0] + (1) Allophana...
2       (2) Pyruvate_c0[c0] + (1) H+_c0[c0] => (1) CO2...
3         (1) Parapyruvate_c0[c0] <=> (2) Pyruvate_c0[c0]
4       (2) H2O2_c0[c0] => (2) H2O_c0[c0] + (1) O2_c0[c0]
                              ...                        
1542    (0.1) Alginate_c0[c0] + (0.8) Pel_c0[c0] + (0....
1543    (50) GDP-D-mannuronate_c0[c0] --> (1) Alginate...
1544    (1) H2O_c0[c0] + (2) NAD_c0[c0] + (1) GDP-D-ma...
1545    (50) UDP-N-acetyl-D-galactosamine_c0[c0] --> (...
1546    (50) GDP-D-mannose_c0[c0] + (50) UDP-glucose_c...
Name: Definition, Length: 1547, dtype: object
处理完成，结果已保存到 14067_rxn_3.xlsx


手动添加交换反应